In [14]:
import mysql.connector
import os
import requests as r
import json
from pandas import DataFrame
from datetime import date
import numpy as np

import holidays

def lookup():
    val_list = []
    validator = open("validation.txt", "r")
    for line in validator:
        val_list.append(line.rstrip())

    mydb = mysql.connector.connect(
      host=val_list[0],
      user=val_list[1],
      passwd=val_list[2],
      database=val_list[3],
      port=val_list[4]
    )

    mycursor = mydb.cursor()
    result_list = []

    sql = """SELECT CONCAT(str_to_date(leavetimes.`dayofservice`,'%Y-%m-%d')," ",SEC_TO_TIME(leavetimes.`actualtime_arr`)) as date,leavetimes.tripid,leavetimes.progrnumber,leavetimes.stoppointid,leavetimes.actualtime_arr, weather.precipIntensity,weather.temperature,trips.lineid, stops.lat, stops.lng FROM leavetimes, weather, trips, stops WHERE weather.time = DATE_FORMAT((CONCAT(str_to_date(leavetimes.`dayofservice`,'%Y-%m-%d')," ",SEC_TO_TIME(leavetimes.`actualtime_arr`))), "%Y-%m-%d %H:00:00") and trips.tripid = leavetimes.tripid and trips.dayofservice = leavetimes.dayofservice and leavetimes.stoppointid = stops.stop_id ORDER BY leavetimes.dayofservice,leavetimes.tripid,leavetimes.progrnumber LIMIT 0,20"""
    mycursor.execute(sql)

    for (i) in mycursor:
        result_list.extend([i])
    frame = DataFrame.from_records(result_list)
    frame.columns = ["Date-Time","Trip_ID","Prog_No","Stop_ID","Time","Precip","Heat","lineid","lat","lng"]
    return frame

DFRAME = lookup()
DFRAME

,Date-Time,Trip_ID,Prog_No,Stop_ID,Time,Precip,Heat,lineid,lat,lng
0,2018-01-01 08:20:29,5955277,1,7347,30029,0.0,5.84,16,53.4280188900,-6.2420269440
1,2018-01-01 08:22:08,5955277,2,3669,30128,0.0,5.84,16,53.4284655600,-6.2350797220
2,2018-01-01 08:23:02,5955277,3,7349,30182,0.0,5.84,16,53.4253238900,-6.2347730560
3,2018-01-01 08:24:15,5955277,4,1631,30255,0.0,5.84,16,53.4216761100,-6.2314400000
4,2018-01-01 08:24:21,5955277,5,1632,30261,0.0,5.84,16,53.4205011100,-6.2331088890
5,2018-01-01 08:24:34,5955277,6,5053,30274,0.0,5.84,16,53.4191288900,-6.2354588890
6,2018-01-01 08:24:47,5955277,7,1633,30287,0.0,5.84,16,53.4177811100,-6.2377238890
7,2018-01-01 08:25:25,5955277,8,1634,30325,0.0,5.84,16,53.4135180600,-6.2388869440
8,2018-01-01 08:25:32,5955277,9,1635,30332,0.0,5.84,16,53.4116969400,-6.2388588890
9,2018-01-01 08:26:55,5955277,10,1636,30415,0.0,5.84,16,53.4082438900,-6.2375869440


In [11]:
ie_holidays = holidays.Ireland()
DFRAME["Holiday"] = 0
DFRAME["Prev_Stop"] = 0
prev_stop = DFRAME.iloc[0]['Stop_ID']
for index, row in DFRAME.iterrows():
    if DFRAME.iloc[index]['Prog_No'] == 1:
        time = DFRAME.iloc[index]['Time']
        DFRAME.set_value(index,"Time",(row["Time"] - time ))
        value = int(row["Date-Time"] in ie_holidays)
        DFRAME.set_value(index,"Holiday",value)
        DFRAME.set_value(index,"Prev_Stop",int(prev_stop))
    else:
        DFRAME.set_value(index,"Time",(row["Time"] - time ))
        value = int(row["Date-Time"] in ie_holidays)
        DFRAME.set_value(index,"Holiday",value)
        DFRAME.set_value(index,"Prev_Stop",int(prev_stop))
    print ("Row " + str(index) + " OK!")
    prev_stop = DFRAME.iloc[index]['Stop_ID']
    time = row["Time"]

E:\Anaconda\envs\env\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
E:\Anaconda\envs\env\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
E:\Anaconda\envs\env\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # This is added back by InteractiveShellApp.init_path()
E:\Anaconda\envs\env\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
E:\Anaconda\envs\env\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and 

Row 0 OK!
Row 1 OK!
Row 2 OK!
Row 3 OK!
Row 4 OK!
Row 5 OK!
Row 6 OK!
Row 7 OK!
Row 8 OK!
Row 9 OK!
Row 10 OK!
Row 11 OK!
Row 12 OK!
Row 13 OK!
Row 14 OK!
Row 15 OK!
Row 16 OK!
Row 17 OK!
Row 18 OK!
Row 19 OK!
Row 20 OK!
Row 21 OK!
Row 22 OK!
Row 23 OK!
Row 24 OK!
Row 25 OK!
Row 26 OK!
Row 27 OK!
Row 28 OK!
Row 29 OK!
Row 30 OK!
Row 31 OK!
Row 32 OK!
Row 33 OK!
Row 34 OK!
Row 35 OK!
Row 36 OK!
Row 37 OK!
Row 38 OK!
Row 39 OK!
Row 40 OK!
Row 41 OK!
Row 42 OK!
Row 43 OK!
Row 44 OK!
Row 45 OK!
Row 46 OK!
Row 47 OK!
Row 48 OK!
Row 49 OK!
Row 50 OK!
Row 51 OK!
Row 52 OK!
Row 53 OK!
Row 54 OK!
Row 55 OK!
Row 56 OK!
Row 57 OK!
Row 58 OK!
Row 59 OK!
Row 60 OK!
Row 61 OK!
Row 62 OK!
Row 63 OK!
Row 64 OK!
Row 65 OK!
Row 66 OK!
Row 67 OK!
Row 68 OK!
Row 69 OK!
Row 70 OK!
Row 71 OK!
Row 72 OK!
Row 73 OK!
Row 74 OK!
Row 75 OK!
Row 76 OK!
Row 77 OK!
Row 78 OK!
Row 79 OK!
Row 80 OK!
Row 81 OK!
Row 82 OK!
Row 83 OK!
Row 84 OK!
Row 85 OK!
Row 86 OK!
Row 87 OK!
Row 88 OK!
Row 89 OK!
Row 90 OK!
Row 91 OK

Row 776 OK!
Row 777 OK!
Row 778 OK!
Row 779 OK!
Row 780 OK!
Row 781 OK!
Row 782 OK!
Row 783 OK!
Row 784 OK!
Row 785 OK!
Row 786 OK!
Row 787 OK!
Row 788 OK!
Row 789 OK!
Row 790 OK!
Row 791 OK!
Row 792 OK!
Row 793 OK!
Row 794 OK!
Row 795 OK!
Row 796 OK!
Row 797 OK!
Row 798 OK!
Row 799 OK!
Row 800 OK!
Row 801 OK!
Row 802 OK!
Row 803 OK!
Row 804 OK!
Row 805 OK!
Row 806 OK!
Row 807 OK!
Row 808 OK!
Row 809 OK!
Row 810 OK!
Row 811 OK!
Row 812 OK!
Row 813 OK!
Row 814 OK!
Row 815 OK!
Row 816 OK!
Row 817 OK!
Row 818 OK!
Row 819 OK!
Row 820 OK!
Row 821 OK!
Row 822 OK!
Row 823 OK!
Row 824 OK!
Row 825 OK!
Row 826 OK!
Row 827 OK!
Row 828 OK!
Row 829 OK!
Row 830 OK!
Row 831 OK!
Row 832 OK!
Row 833 OK!
Row 834 OK!
Row 835 OK!
Row 836 OK!
Row 837 OK!
Row 838 OK!
Row 839 OK!
Row 840 OK!
Row 841 OK!
Row 842 OK!
Row 843 OK!
Row 844 OK!
Row 845 OK!
Row 846 OK!
Row 847 OK!
Row 848 OK!
Row 849 OK!
Row 850 OK!
Row 851 OK!
Row 852 OK!
Row 853 OK!
Row 854 OK!
Row 855 OK!
Row 856 OK!
Row 857 OK!
Row 858 OK!
Row 

Row 1668 OK!
Row 1669 OK!
Row 1670 OK!
Row 1671 OK!
Row 1672 OK!
Row 1673 OK!
Row 1674 OK!
Row 1675 OK!
Row 1676 OK!
Row 1677 OK!
Row 1678 OK!
Row 1679 OK!
Row 1680 OK!
Row 1681 OK!
Row 1682 OK!
Row 1683 OK!
Row 1684 OK!
Row 1685 OK!
Row 1686 OK!
Row 1687 OK!
Row 1688 OK!
Row 1689 OK!
Row 1690 OK!
Row 1691 OK!
Row 1692 OK!
Row 1693 OK!
Row 1694 OK!
Row 1695 OK!
Row 1696 OK!
Row 1697 OK!
Row 1698 OK!
Row 1699 OK!
Row 1700 OK!
Row 1701 OK!
Row 1702 OK!
Row 1703 OK!
Row 1704 OK!
Row 1705 OK!
Row 1706 OK!
Row 1707 OK!
Row 1708 OK!
Row 1709 OK!
Row 1710 OK!
Row 1711 OK!
Row 1712 OK!
Row 1713 OK!
Row 1714 OK!
Row 1715 OK!
Row 1716 OK!
Row 1717 OK!
Row 1718 OK!
Row 1719 OK!
Row 1720 OK!
Row 1721 OK!
Row 1722 OK!
Row 1723 OK!
Row 1724 OK!
Row 1725 OK!
Row 1726 OK!
Row 1727 OK!
Row 1728 OK!
Row 1729 OK!
Row 1730 OK!
Row 1731 OK!
Row 1732 OK!
Row 1733 OK!
Row 1734 OK!
Row 1735 OK!
Row 1736 OK!
Row 1737 OK!
Row 1738 OK!
Row 1739 OK!
Row 1740 OK!
Row 1741 OK!
Row 1742 OK!
Row 1743 OK!
Row 1744 OK!

Row 2582 OK!
Row 2583 OK!
Row 2584 OK!
Row 2585 OK!
Row 2586 OK!
Row 2587 OK!
Row 2588 OK!
Row 2589 OK!
Row 2590 OK!
Row 2591 OK!
Row 2592 OK!
Row 2593 OK!
Row 2594 OK!
Row 2595 OK!
Row 2596 OK!
Row 2597 OK!
Row 2598 OK!
Row 2599 OK!
Row 2600 OK!
Row 2601 OK!
Row 2602 OK!
Row 2603 OK!
Row 2604 OK!
Row 2605 OK!
Row 2606 OK!
Row 2607 OK!
Row 2608 OK!
Row 2609 OK!
Row 2610 OK!
Row 2611 OK!
Row 2612 OK!
Row 2613 OK!
Row 2614 OK!
Row 2615 OK!
Row 2616 OK!
Row 2617 OK!
Row 2618 OK!
Row 2619 OK!
Row 2620 OK!
Row 2621 OK!
Row 2622 OK!
Row 2623 OK!
Row 2624 OK!
Row 2625 OK!
Row 2626 OK!
Row 2627 OK!
Row 2628 OK!
Row 2629 OK!
Row 2630 OK!
Row 2631 OK!
Row 2632 OK!
Row 2633 OK!
Row 2634 OK!
Row 2635 OK!
Row 2636 OK!
Row 2637 OK!
Row 2638 OK!
Row 2639 OK!
Row 2640 OK!
Row 2641 OK!
Row 2642 OK!
Row 2643 OK!
Row 2644 OK!
Row 2645 OK!
Row 2646 OK!
Row 2647 OK!
Row 2648 OK!
Row 2649 OK!
Row 2650 OK!
Row 2651 OK!
Row 2652 OK!
Row 2653 OK!
Row 2654 OK!
Row 2655 OK!
Row 2656 OK!
Row 2657 OK!
Row 2658 OK!

Row 3429 OK!
Row 3430 OK!
Row 3431 OK!
Row 3432 OK!
Row 3433 OK!
Row 3434 OK!
Row 3435 OK!
Row 3436 OK!
Row 3437 OK!
Row 3438 OK!
Row 3439 OK!
Row 3440 OK!
Row 3441 OK!
Row 3442 OK!
Row 3443 OK!
Row 3444 OK!
Row 3445 OK!
Row 3446 OK!
Row 3447 OK!
Row 3448 OK!
Row 3449 OK!
Row 3450 OK!
Row 3451 OK!
Row 3452 OK!
Row 3453 OK!
Row 3454 OK!
Row 3455 OK!
Row 3456 OK!
Row 3457 OK!
Row 3458 OK!
Row 3459 OK!
Row 3460 OK!
Row 3461 OK!
Row 3462 OK!
Row 3463 OK!
Row 3464 OK!
Row 3465 OK!
Row 3466 OK!
Row 3467 OK!
Row 3468 OK!
Row 3469 OK!
Row 3470 OK!
Row 3471 OK!
Row 3472 OK!
Row 3473 OK!
Row 3474 OK!
Row 3475 OK!
Row 3476 OK!
Row 3477 OK!
Row 3478 OK!
Row 3479 OK!
Row 3480 OK!
Row 3481 OK!
Row 3482 OK!
Row 3483 OK!
Row 3484 OK!
Row 3485 OK!
Row 3486 OK!
Row 3487 OK!
Row 3488 OK!
Row 3489 OK!
Row 3490 OK!
Row 3491 OK!
Row 3492 OK!
Row 3493 OK!
Row 3494 OK!
Row 3495 OK!
Row 3496 OK!
Row 3497 OK!
Row 3498 OK!
Row 3499 OK!
Row 3500 OK!
Row 3501 OK!
Row 3502 OK!
Row 3503 OK!
Row 3504 OK!
Row 3505 OK!

KeyboardInterrupt: 

In [4]:


for index, row in DFRAME.iterrows():
    )

C:\Users\Corma\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


In [5]:

prev_stop = DFRAME.iloc[0]['Stop_ID']
for index, row in DFRAME.iterrows():
    DFRAME.set_value(index,"Prev_Stop",int(prev_stop))
    prev_stop = DFRAME.iloc[index]['Stop_ID']



C:\Users\Corma\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


In [6]:
DFRAME.tail()

,Date-Time,Trip_ID,Prog_No,Stop_ID,Time,Weather,lineid,Holiday,Prev_Stop
4102327,2018-12-31 22:34:16,8591669,66,2964,64,Overcast,145,0,2963
4102328,2018-12-31 22:36:03,8591669,67,4175,107,Overcast,145,0,2964
4102329,2018-12-31 22:36:40,8591669,68,5091,37,Overcast,145,0,4175
4102330,2018-12-31 22:37:19,8591669,69,5092,39,Overcast,145,0,5091
4102331,2018-12-31 22:38:00,8591669,70,7574,41,Overcast,145,0,5092


In [7]:
DFRAME.to_csv("Transformed145leavetimes.csv")

In [9]:
stop_pairs = []
for index, row in DFRAME.iterrows():
    temp_stop = str(DFRAME.iloc[index]['Stop_ID']) + " " + str(DFRAME.iloc[index]['Prev_Stop'])
    if temp_stop in stop_pairs:
        continue
    else:
        stop_pairs.append(temp_stop)
    
    

In [10]:
print(stop_pairs)

['4320 4320', '1476 4320', '7453 1476', '1478 7453', '1479 1478', '7622 1479', '334 7622', '406 334', '747 406', '842 747', '845 842', '846 845', '847 846', '848 847', '2795 848', '756 2795', '757 756', '758 757', '759 758', '760 759', '761 760', '762 761', '763 762', '2007 763', '2008 2007', '2009 2008', '2010 2009', '435 2010', '7353 435', '4571 7353', '2013 4571', '2014 2013', '4636 2014', '2016 4636', '2015 2016', '2017 2015', '3259 2017', '7361 3259', '7362 7361', '3129 7362', '7638 3129', '3130 7638', '3131 3130', '3132 3131', '3133 3132', '3134 3133', '3135 3134', '3136 3135', '3138 3136', '3139 3138', '4124 3139', '4125 4124', '4126 4125', '4127 4126', '4128 4127', '4129 4128', '4130 4129', '4131 4130', '7294 4131', '4132 7294', '4171 4132', '4172 4171', '4173 4172', '2962 4173', '2963 2962', '2964 2963', '4175 2964', '5091 4175', '5092 5091', '7574 5092', '7574 7574', '4177 7574', '4178 4177', '4179 4178', '2993 4179', '2994 2993', '4180 2994', '4181 4180', '4182 4181', '4151 

In [11]:
with open('145pairs.txt', 'w') as f:
    for item in stop_pairs:
        f.write("%s\n" % item)

In [12]:
DFRAME

,Date-Time,Trip_ID,Prog_No,Stop_ID,Time,Precip,Heat,lineid,Holiday,Prev_Stop
0,2018-01-01 08:20:29,5955277,1,7347,0,0.0,5.84,16,1,7347
1,2018-01-01 08:22:08,5955277,2,3669,99,0.0,5.84,16,1,7347
2,2018-01-01 08:23:02,5955277,3,7349,-243,0.0,5.84,16,1,3669
3,2018-01-01 08:24:15,5955277,4,1631,208,0.0,5.84,16,1,7349
4,2018-01-01 08:24:21,5955277,5,1632,-150,0.0,5.84,16,1,1631
5,2018-01-01 08:24:34,5955277,6,5053,160,0.0,5.84,16,1,1632
6,2018-01-01 08:24:47,5955277,7,1633,-81,0.0,5.84,16,1,5053
7,2018-01-01 08:25:25,5955277,8,1634,32,0.0,5.84,16,1,1633
8,2018-01-01 08:25:32,5955277,9,1635,-81,0.0,5.84,16,1,1634
9,2018-01-01 08:26:55,5955277,10,1636,163,0.0,5.84,16,1,1635
